### Automatic model selection

Test notebook for automatic model selection.

In [1]:
import datetime
import logging
import sys
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
import keras.backend as K

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/media/controlslab/DATA/Projects')

import ann_framework.aux_functions as aux_functions

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
import fetch_to_keras
#from CMAPSAuxFunctions import TrainValTensorBoard

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression, SequenceTunableModelClassification

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPSS import CMAPSSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler
from ann_framework.data_handlers.data_handler_CIFAR10 import CIFAR10DataHandler

learningRate_scheduler = LearningRateScheduler(aux_functions.step_decay)

size_scaler = 0.8

#Use same configuration for all experiments, just change some of the parameters

#Define some random paramaters for the creation of the configuration, this will change for each test model
architecture_type = Layers.FullyConnected
problem_type = 2  #1 for regression, 2 for classification
output_shape = 10 #If regression applies, number of classes
input_shape = (784,)

config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size=5, 
                       tournament_size=3, max_similar=3, epochs=20, cross_val=0.2, size_scaler=size_scaler,
                       max_generations=10, binary_selection=True, mutation_ratio=0.4, 
                       similarity_threshold=0.2, more_layers_prob=0.8)



Using TensorFlow backend.


### Given a model, get the compiled model

In [2]:
def get_compiled_model(model, problem_type, optimizer_params=[]):
    """Obtain a keras compiled model"""
    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001, beta_1=0.5)
    
    if problem_type == 1:
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    
    return model


def create_tunable_model(model_genotype, problem_type, input_shape, data_handler, model_number):
    
    K.clear_session()
    
    model = fetch_to_keras.decode_genotype(model_genotype, problem_type, input_shape, 1)
    
    model = get_compiled_model(model, problem_type, optimizer_params=[])
    
    if problem_type == 1:
        tModel = SequenceTunableModelRegression('ModelReg_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
    else:
        tModel = SequenceTunableModelClassification('ModelClass_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
        
    return tModel

### Load cmaps data handler

In [3]:
def cmaps_dhandler():

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 25
    window_stride = 1
    max_rul = 130

    dHandler_cmaps = CMAPSSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)

    input_shape = (len(selected_features)*window_size, )

    return dHandler_cmaps, input_shape

### Function to save top models

In [4]:
def save_best_models(best_models_list, global_best_model_index, saveto, input_shape, data_handler, 
                     problem_type=1, data_scaler=None, train_epochs=100, metrics=[], round=0):
    
    n_models = len(best_models_list)
    
    for ind_model, i in zip(best_models_list, range(n_models)):
        
        tModel = create_tunable_model(ind_model.stringModel, problem_type, input_shape, data_handler, i)
        kmodel = tModel.model
        model_json = kmodel.to_json()
        
        #Save model's architecture
        string_append = str(i) if i != global_best_model_index else 'global'
        with open(saveto+"bestModel_"+string_append+".json", "w") as json_file:
            json_file.write(model_json)
            
    #Train the global best, model has to be recompiled
    ind_model = best_models_list[global_best_model_index]
    tModel = create_tunable_model(ind_model.stringModel, problem_type, input_shape, data_handler, n_models)
    
    print(tModel.model.summary())
    print(tModel.data_handler)
    
    if tModel.data_handler.data_scaler != None:
        print("Using data handler scaler")
    elif tModel.data_scaler != None:
        print("Using tModel scaler (Overriding data handler scaler)")
    else:
        print("No data scaling used")
    
    if data_scaler != None:
        tModel.data_handler.data_scaler = None
        tModel.data_scaler = data_scaler
        
    tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
    tModel.print_data()
    tModel.epochs = train_epochs

    tModel.train_model(verbose=1)
    
    tModel.evaluate_model(metrics, round=round)
    
    kmodel = tModel.model
            
    # serialize weights to HDF5
    kmodel.save_weights(saveto+"bestModel_global.h5")
    
    print("Saved models for dataset 1 to disk")
        

### Test on MNIST

In [ ]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 2  #1 for regression, 2 for classification
output_shape = 10 #If regression applies, number of classes
input_shape = (784,)

"""
pop_size = 5
tournament_size = 3
max_similar = 3
total_experiments = 5
count_experiments = 0
unroll = True
"""
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None
global_best_index = 0

scaler = None

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_mnist_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#mnist datahandler
dHandler_mnist = MNISTDataHandler()

config.architecture_type = architecture_type
config.problem_type = problem_type
config.input_shape = input_shape
config.output_shape = output_shape

"""
config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, 
                       tournament_size, max_similar, epochs=20, cross_val=0.2, size_scaler=size_scaler,
                       max_generations=10, binary_selection=True, mutation_ratio=0.4, 
                       similarity_threshold=0.2, more_layers_prob=0.8)
"""

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    
    best = automatic_model_selection.run_experiment(config, dHandler_mnist, count_experiments + 1, unroll=unroll,
                                                    learningRate_scheduler=learningRate_scheduler, 
                                                    tModel_scaler=scaler, verbose_data=0)

    best.individual_label = count_experiments
    
    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best
            global_best_index = count_experiments

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
automatic_model_selection.print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

Launching experiment 1

Generation 1
launch new
True
gen similar
False
Fetching model 0 to keras
Evaluating model 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 264)               207240    
_________________________________________________________________
dropout_1 (Dropout)          (None, 264)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 952)               252280    
_________________________________________________________________
dense_2 (Dense)              (None, 56)                53368     
_________________________________________________________________
dense_3 (Dense)              (None, 312)               17784     
_________________________________________________________________
dense_4 (Dense)              (None, 448)               140224    
__________________________

/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 1s 23us/step - loss: 2.0766 - acc: 0.1775 - val_loss: 1.5755 - val_acc: 0.4146
Epoch 2/20
48000/48000 [==============================] - 0s 6us/step - loss: 1.2327 - acc: 0.5149 - val_loss: 0.9190 - val_acc: 0.6874
Epoch 3/20
48000/48000 [==============================] - 0s 6us/step - loss: 0.8403 - acc: 0.7125 - val_loss: 0.6527 - val_acc: 0.7875
Epoch 4/20
48000/48000 [==============================] - 0s 6us/step - loss: 0.6427 - acc: 0.8052 - val_loss: 0.5551 - val_acc: 0.8372
Epoch 5/20
48000/48000 [==============================] - 0s 7us/step - loss: 0.5353 - acc: 0.8477 - val_loss: 0.4785 - val_acc: 0.8672
Epoch 6/20
48000/48000 [==============================] - 0s 6us/step - loss: 0.4572 - acc: 0.8750 - val_loss: 0.3823 - val_acc: 0.8980
Epoch 7/20
48000/48000 [==============================] - 0s 7us/step - loss: 0.3992 - acc: 0.8913 - val_loss: 0.363

48000/48000 [==============================] - 1s 12us/step - loss: 2.3012 - acc: 0.1118 - val_loss: 2.3014 - val_acc: 0.1127
Epoch 19/20
48000/48000 [==============================] - 1s 12us/step - loss: 2.3012 - acc: 0.1123 - val_loss: 2.3014 - val_acc: 0.1127
Epoch 20/20
12000/12000 [==============================] - 0s 24us/step
Fetching model 2 to keras
Evaluating model 2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 96)                75360     
_________________________________________________________________
dense_1 (Dense)              (None, 864)               83808     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                8650      
Total params: 167,818
Trainable params: 167,818
Non-trainable params: 0
_________________________________________________________________
None
training with 

training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 1s 13us/step - loss: 1.3899 - acc: 0.4894 - val_loss: 0.8055 - val_acc: 0.7068
Epoch 2/20
48000/48000 [==============================] - 0s 8us/step - loss: 0.6452 - acc: 0.7871 - val_loss: 0.5812 - val_acc: 0.8117
Epoch 3/20
48000/48000 [==============================] - 0s 8us/step - loss: 0.4648 - acc: 0.8609 - val_loss: 0.3922 - val_acc: 0.8843
Epoch 4/20
48000/48000 [==============================] - 0s 8us/step - loss: 0.3603 - acc: 0.8940 - val_loss: 0.3256 - val_acc: 0.9015
Epoch 5/20
48000/48000 [==============================] - 0s 8us/step - loss: 0.2832 - acc: 0.9170 - val_loss: 0.2702 - val_acc: 0.9193
Epoch 6/20
48000/48000 [==============================] - 0s 8us/step - loss: 0.2424 - acc: 0.9275 - val_loss: 0.2527 - val_acc: 0.9253
Epoch 7/20
48000/48000 [==============================] - 0s 8us/step - loss: 0.2043 - acc: 0.9391 - val_loss: 0.218

...

training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 0s 7us/step - loss: 0.5241 - acc: 0.8589 - val_loss: 0.3109 - val_acc: 0.9155
Epoch 2/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.2839 - acc: 0.9193 - val_loss: 0.2619 - val_acc: 0.9249
Epoch 3/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.2378 - acc: 0.9320 - val_loss: 0.2240 - val_acc: 0.9365
Epoch 4/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.2008 - acc: 0.9435 - val_loss: 0.1929 - val_acc: 0.9453
Epoch 5/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.1717 - acc: 0.9520 - val_loss: 0.1685 - val_acc: 0.9522
Epoch 6/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.1480 - acc: 0.9579 - val_loss: 0.1517 - val_acc: 0.9566
Epoch 7/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.1287 - acc: 0.9637 - val_loss: 0.1352

48000/48000 [==============================] - 0s 4us/step - loss: 0.1526 - acc: 0.9571 - val_loss: 0.1634 - val_acc: 0.9512
Epoch 10/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.1416 - acc: 0.9609 - val_loss: 0.1542 - val_acc: 0.9543
Epoch 11/20
48000/48000 [==============================] - 0s 5us/step - loss: 0.1319 - acc: 0.9634 - val_loss: 0.1511 - val_acc: 0.9558
Epoch 12/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.1227 - acc: 0.9668 - val_loss: 0.1413 - val_acc: 0.9585
Epoch 13/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.1146 - acc: 0.9689 - val_loss: 0.1365 - val_acc: 0.9599
Epoch 14/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.1076 - acc: 0.9709 - val_loss: 0.1321 - val_acc: 0.9613
Epoch 15/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.1012 - acc: 0.9719 - val_loss: 0.1282 - val_acc: 0.9616
Epoch 16/20
48000/48000 [============================

48000/48000 [==============================] - 0s 4us/step - loss: 0.2429 - acc: 0.9284 - val_loss: 0.2198 - val_acc: 0.9376
Epoch 14/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.2385 - acc: 0.9298 - val_loss: 0.2143 - val_acc: 0.9383
Epoch 15/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.2351 - acc: 0.9310 - val_loss: 0.2095 - val_acc: 0.9392
Epoch 16/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.2289 - acc: 0.9331 - val_loss: 0.2076 - val_acc: 0.9401
Epoch 17/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.2250 - acc: 0.9332 - val_loss: 0.2053 - val_acc: 0.9402
Epoch 18/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.2234 - acc: 0.9337 - val_loss: 0.1989 - val_acc: 0.9432
Epoch 19/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.2187 - acc: 0.9343 - val_loss: 0.1983 - val_acc: 0.9420
Epoch 20/20
12000/12000 [============================

48000/48000 [==============================] - 0s 10us/step - loss: 2.3155 - acc: 0.1049 - val_loss: 2.3143 - val_acc: 0.1010
Epoch 6/20
48000/48000 [==============================] - 0s 10us/step - loss: 2.3144 - acc: 0.1004 - val_loss: 2.3179 - val_acc: 0.1140
Epoch 7/20
48000/48000 [==============================] - 0s 10us/step - loss: 2.3139 - acc: 0.1031 - val_loss: 2.3097 - val_acc: 0.1140
Epoch 8/20
48000/48000 [==============================] - 0s 10us/step - loss: 2.3128 - acc: 0.1055 - val_loss: 2.3299 - val_acc: 0.1022
Epoch 9/20
48000/48000 [==============================] - 0s 10us/step - loss: 2.3146 - acc: 0.1034 - val_loss: 2.3125 - val_acc: 0.1140
Epoch 10/20
48000/48000 [==============================] - 0s 10us/step - loss: 2.3154 - acc: 0.1010 - val_loss: 2.3035 - val_acc: 0.1140
Epoch 11/20
48000/48000 [==============================] - 0s 10us/step - loss: 2.3137 - acc: 0.1029 - val_loss: 2.3188 - val_acc: 0.0974
Epoch 12/20
48000/48000 [=========================

Epoch 11/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.0746 - acc: 0.9795 - val_loss: 0.1087 - val_acc: 0.9700
Epoch 12/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.0668 - acc: 0.9821 - val_loss: 0.1087 - val_acc: 0.9691
Epoch 13/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.0592 - acc: 0.9844 - val_loss: 0.1017 - val_acc: 0.9708
Epoch 14/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.0531 - acc: 0.9858 - val_loss: 0.0951 - val_acc: 0.9729
Epoch 15/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.0470 - acc: 0.9882 - val_loss: 0.0926 - val_acc: 0.9731
Epoch 16/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.0422 - acc: 0.9891 - val_loss: 0.0934 - val_acc: 0.9739
Epoch 17/20
48000/48000 [==============================] - 0s 4us/step - loss: 0.0378 - acc: 0.9909 - val_loss: 0.0877 - val_acc: 0.9752
Epoch 18/20
48000/48000 [================

In [ ]:
save_best_models(global_best_list, global_best_index, 'best_models/mnist/alpha{}/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dHandler_mnist, problem_type=problem_type, train_epochs=100)

### Test on CIFAR10

In [ ]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 2  #1 for regression, 2 for classification
output_shape = 10 #If regression applies, number of classes
input_shape = (3072,)
"""
pop_size = 5
tournament_size = 3
max_similar = 3
"""
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None
global_best_index = 0

scaler = None

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_cifar10_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#mnist datahandler
dHandler_cifar = CIFAR10DataHandler()

"""
config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, max_similar, 
                       epochs=5, cross_val=0.2, size_scaler=size_scaler, max_generations=10, binary_selection=True, 
                       mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)
"""

config.architecture_type = architecture_type
config.problem_type = problem_type
config.input_shape = input_shape
config.output_shape = output_shape

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    
    best = automatic_model_selection.run_experiment(config, dHandler_cifar, count_experiments + 1, unroll=unroll,
                                                    learningRate_scheduler=learningRate_scheduler, 
                                                    tModel_scaler=scaler, verbose_data=0)
    
    best.individual_label = count_experiments

    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best
            global_best_index = count_experiments

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
automatic_model_selection.print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

In [ ]:
save_best_models(global_best_list, global_best_index, 'best_models/cifar10/alpha{}/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dHandler_cifar, problem_type=problem_type, train_epochs=100)

### Test on CMAPSS

In [ ]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 1  #1 for regression, 2 for classification
output_shape = 1 #If regression applies, number of classes
input_shape = None

"""
pop_size = 5
tournament_size = 3
max_similar = 3
"""
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None
global_best_index = 0

scaler = MinMaxScaler(feature_range=(-1, 1))

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_cmaps_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#cmaps datahandler
dhandler_cmaps, input_shape = cmaps_dhandler()

"""
config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, 
                       max_similar, epochs=5, cross_val=0.2, size_scaler=size_scaler, max_generations=10, 
                       binary_selection=True, mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)
"""

config.architecture_type = architecture_type
config.problem_type = problem_type
config.input_shape = input_shape
config.output_shape = output_shape

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    
    best = automatic_model_selection.run_experiment(config, dhandler_cmaps, count_experiments + 1, unroll=unroll,
                                                    learningRate_scheduler=learningRate_scheduler, 
                                                    tModel_scaler=scaler, verbose_data=0)
    
    best.individual_label = count_experiments

    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best
            global_best_index = count_experiments

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
automatic_model_selection.print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

In [ ]:
save_best_models(global_best_list, global_best_index, 'best_models/cmapss/alpha{}/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dhandler_cmaps, problem_type=problem_type, train_epochs=100, 
                 data_scaler=scaler, metrics=['rhs', 'rmse'], round=2)